In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from breeze_connect import BreezeConnect
import datetime as dt
import base64 
import socketio



In [2]:
Key = '73=63fT44781tY4C42L0i3491201v23A'
Secret = '11331XF5fm6c5&Y3K2Q99lp28347480+'
api_session = '37327151'

In [3]:

# Initialize SDK
breeze = BreezeConnect(api_key=Key)

# Obtain your session key from https://api.icicidirect.com/apiuser/login?api_key=YOUR_API_KEY
# Incase your api-key has special characters(like +,=,!) then encode the api key before using in the url as shown below.

#print("https://api.icicidirect.com/apiuser/login?api_key="+urllib.parse.quote_plus("your_api_key"))

# Generate Session
breeze.generate_session(api_secret=Secret,
                        session_token=api_session)

# Generate ISO8601 Date/DateTime String
import datetime
iso_date_string = datetime.datetime.strptime("07/12/2023","%d/%m/%Y").isoformat()[:10] + 'T05:30:00.000Z'
iso_date_time_string = datetime.datetime.strptime("07/12/2023 23:59:59","%d/%m/%Y %H:%M:%S").isoformat()[:19] + '.000Z'

In [4]:
stock = 'CNXBAN'
s_type = 'futures'
exp = '27-Mar-2024'

In [5]:
breeze.ws_connect()

def on_ticks(ticks):
    print("Ticks: {}".format(ticks))

breeze.on_ticks = on_ticks

breeze.subscribe_feeds(exchange_code="NFO", stock_code=stock, product_type=s_type, expiry_date=exp)


{'message': 'Stock CNXBAN subscribed successfully'}

Ticks: {'symbol': '4.1!36611', 'open': 46475, 'last': 46251, 'high': 46699, 'low': 46002, 'change': -0.43, 'bPrice': 46250.3, 'bQty': 120, 'sPrice': 46258, 'sQty': 30, 'ltq': 15, 'avgPrice': 46364.32, 'quotes': 'Quotes Data', 'OI': '2100195', 'CHNGOI': '', 'ttq': 1434465, 'totalBuyQt': 106890, 'totalSellQ': 170415, 'ttv': '6650.8C', 'trend': '', 'lowerCktLm': 41804.15, 'upperCktLm': 51094, 'ltt': 'Wed Mar 20 11:27:32 2024', 'close': 46449.05, 'exchange': 'NSE Futures & Options', 'stock_name': 'NIFTY BANK', 'product_type': 'Futures', 'expiry_date': '27-Mar-2024'}
Ticks: {'symbol': '4.2!36611', 'time': 'Wed Mar 20 11:27:32 2024', 'depth': [{'BestBuyRate-1': 46250.3, 'BestBuyQty-1': 120, 'BuyNoOfOrders-1': 1, 'BuyFlag-1': '', 'BestSellRate-1': 46258, 'BestSellQty-1': 30, 'SellNoOfOrders-1': 2, 'SellFlag-1': ''}, {'BestBuyRate-2': 46250.25, 'BestBuyQty-2': 45, 'BuyNoOfOrders-2': 1, 'BuyFlag-2': '', 'BestSellRate-2': 46258.25, 'BestSellQty-2': 15, 'SellNoOfOrders-2': 1, 'SellFlag-2': ''}, {

In [5]:
old_price_long = 0
old_price_short = 0
target_prencantage = 0.1
stop_loss_precentage = 1 
long_hold = False
short_hold = False
trade_book = []
qty = 15
brokerage = 20
stt = 0.025/100
transaction_charge = 0.000325/100
stamp = 0.003/100
sebi_charges = 10

In [6]:
breeze.ws_connect()

def on_ticks(ticks):
    global old_price_short,old_price_short,target_prencantage,stop_loss_precentage,long_hold,short_hold,trade_book,qty,brokerage,stt,transaction_charge,stamp,sebi_charges
    print(ticks)
    date = ticks['datetime']
    current_price = float(ticks['close'])
    if not long_hold:
        print('Got False going long')
        long_hold = True
        print(f'Buying at a price {current_price}. total buy price {qty*current_price} \n ')
        old_price_long = current_price
        order_value = qty*current_price
        tc = order_value*transaction_charge
        stamp_rs = order_value*stamp
        charges = brokerage+stamp_rs+brokerage+sebi_charges+tc
        trade_book.append({'date':date,'type':'long','price':current_price,'profit':0,'charges':charges,'profit_after_charges':-charges})


    
    pre_incrase = ((current_price-old_price_long)/old_price_long)*100
    if pre_incrase >= target_prencantage and long_hold:
        long_hold = False
        profit_long = current_price-old_price_long
        print(f'selling at a price {current_price}. profit = {(current_price-old_price_long)*qty} \n')
        order_value = qty*current_price
        tc = order_value*transaction_charge
        stt_rs = order_value*stt
        charges = brokerage+stt_rs+brokerage+sebi_charges
        trade_book.append({'date':date,'type':'exit long','price':current_price,'profit':profit_long*qty,'charges':charges,'profit_after_charges':(profit_long*qty)-charges})

    
    if not short_hold:
        print('Got False going short')
        short_hold = True
        print(f'Buying at a price {current_price}. total buy price {qty*current_price} \n ')
        old_price_short = current_price
        order_value = qty*current_price
        tc = order_value*transaction_charge
        stt_rs = order_value*stt
        charges = brokerage+stt_rs+brokerage+sebi_charges
        trade_book.append({'date':date,'type':'short','price':current_price,'profit':0,'charges':charges,'profit_after_charges':0-charges})

    
    pre_dec = ((-current_price+old_price_short)/old_price_short)*100
    if pre_dec >= target_prencantage and short_hold:
        short_hold = False
        profit_short = -current_price+old_price_short
        print(f'selling at a price {current_price}. profit = {(-current_price+old_price_short)*qty} \n')
        order_value = qty*current_price
        tc = order_value*transaction_charge
        stamp_rs = order_value*stamp
        charges = brokerage+stamp_rs+brokerage+sebi_charges
        trade_book.append({'date':date,'type':'exit short','price':current_price,'profit':profit_short*qty,'charges':charges,'profit_after_charges':(profit_long*qty)-charges})
    '''
    if index.hour==15 and index.minute==29:
        if long_hold:
            long_hold = False
            profit_long = current_price-old_price
            print(f'Ending Trade for day {current_price}. profit = {(current_price-old_price)*qty} \n')
            order_value = qty*current_price
            tc = order_value*transaction_charge
            stt_rs = order_value*stt
            charges = brokerage+stt_rs+brokerage+sebi_charges
            trade_book.append({'date':index,'type':'exit long','price':current_price,'profit':profit_long*qty,'charges':charges,'profit_after_charges':(profit_long*qty)-charges})


        if short_hold:
            short_hold = False
            profit_short = -current_price+old_price
            print(f'Ending Short at a price {current_price}. profit = {(-current_price+old_price)*qty} \n')
            order_value = qty*current_price
            tc = order_value*transaction_charge
            stamp_rs = order_value*stamp
            charges = brokerage+stamp_rs+brokerage+sebi_charges
            trade_book.append({'date':index,'type':'exit short','price':current_price,'profit':profit_short*qty,'charges':charges,'profit_after_charges':(profit_long*qty)-charges})
            
    
        break'''

breeze.on_ticks = on_ticks

breeze.subscribe_feeds(exchange_code="NFO", stock_code=stock, product_type=s_type, expiry_date=exp,interval='1minute')


{'message': 'Stock CNXBAN subscribed successfully'}

{'interval': '1minute', 'exchange_code': 'NFO', 'stock_code': 'CNXBAN', 'expiry_date': '27-Mar-2024', 'low': '46240.35', 'high': '46271.0', 'open': '46241.7', 'close': '46271.0', 'volume': '2955', 'oi': '2100195', 'datetime': '2024-03-20 11:27:00'}
Got False going long
Buying at a price 46271.0. total buy price 694065.0 
 
Got False going long
Buying at a price 46271.0. total buy price 694065.0 
 


Exception in thread Thread-63 (_handle_eio_message):
Traceback (most recent call last):
  File "c:\Program Files\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Program Files\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\socketio\client.py", line 489, in _handle_eio_message
    self._handle_event(pkt.namespace, pkt.id, pkt.data)
  File "C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\socketio\client.py", line 361, in _handle_event
    r = self._trigger_event(data[0], namespace, *data[1:])
  File "C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\socketio\client.py", line 409, in _trigger_event
    return self.handlers[namespace][event](*args)
  File "C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\breeze_connect\breeze_connect.py", line 75, in on_ohlc_stream
    self.breeze.on_ticks(data)
  File

{'interval': '1minute', 'exchange_code': 'NFO', 'stock_code': 'CNXBAN', 'expiry_date': '27-Mar-2024', 'low': '46248.0', 'high': '46271.6', 'open': '46271.6', 'close': '46248.0', 'volume': '2415', 'oi': '2099595', 'datetime': '2024-03-20 11:28:00'}


Exception in thread Thread-113 (_handle_eio_message):
Traceback (most recent call last):
  File "c:\Program Files\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Program Files\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\socketio\client.py", line 489, in _handle_eio_message
    self._handle_event(pkt.namespace, pkt.id, pkt.data)
  File "C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\socketio\client.py", line 361, in _handle_event
    r = self._trigger_event(data[0], namespace, *data[1:])
  File "C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\socketio\client.py", line 409, in _trigger_event
    return self.handlers[namespace][event](*args)
  File "C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\breeze_connect\breeze_connect.py", line 75, in on_ohlc_stream
    self.breeze.on_ticks(data)
  Fil

{'interval': '1minute', 'exchange_code': 'NFO', 'stock_code': 'CNXBAN', 'expiry_date': '27-Mar-2024', 'low': '46243.5', 'high': '46275.0', 'open': '46248.0', 'close': '46268.7', 'volume': '1965', 'oi': '2099595', 'datetime': '2024-03-20 11:29:00'}


Exception in thread Thread-156 (_handle_eio_message):
Traceback (most recent call last):
  File "c:\Program Files\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Program Files\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\socketio\client.py", line 489, in _handle_eio_message
    self._handle_event(pkt.namespace, pkt.id, pkt.data)
  File "C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\socketio\client.py", line 361, in _handle_event
    r = self._trigger_event(data[0], namespace, *data[1:])
  File "C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\socketio\client.py", line 409, in _trigger_event
    return self.handlers[namespace][event](*args)
  File "C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\breeze_connect\breeze_connect.py", line 75, in on_ohlc_stream
    self.breeze.on_ticks(data)
  Fil

{'interval': '1minute', 'exchange_code': 'NFO', 'stock_code': 'CNXBAN', 'expiry_date': '27-Mar-2024', 'low': '46262.25', 'high': '46275.0', 'open': '46268.7', 'close': '46262.3', 'volume': '1635', 'oi': '2099595', 'datetime': '2024-03-20 11:30:00'}


In [1]:
breeze.ws_disconnect()

NameError: name 'breeze' is not defined